# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация
Дата выдачи: 15.03.2020

Мягкий дедлайн: 02:59MSK 30.03.2020

Жесткий дедлайн: 23:59MSK 03.04.2020

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Kitchen Sinks для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [2]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0],-1)
x_test = x_test_pics.reshape(x_test_pics.shape[0],-1)

Using TensorFlow backend.


4423680/4422102 [==============================] - 1s 0us/step


In [0]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from statistics import median
from math import sqrt
from math import pi
from sklearn.metrics import accuracy_score

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Желательно в виде класса с методами fit() и predict().

Что должно быть в вашей реализации:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [0]:
from sklearn.base import BaseEstimator


class Taskone():

    def __init__(self, new_dim = 50, n_features = 1000, pca = True, svm = False, kernel='linear', regularisation=1, tolerance=1e-3, max_iter=10000):

        self.new_dim = new_dim
        self.n_features = n_features
        self.pca = pca
        self.svm = svm
        self.model = None 
        self.pcamodel = None
        self.B = []
        self.W = []
        self.kernel = kernel
        self.regularisation = regularisation
        self.tolerance = tolerance
        self.max_iter = max_iter

    
    def fit(self, X, y):

        #понижение размерности
        if self.pca == True:
            self.pcamodel = PCA(n_components=self.new_dim)
            self.pcamodel.fit(X)
            X = self.pcamodel.transform(X)

        #подсчет сигмы
        pares = []
        limit = min(1000, X.shape[0])   #если наблюдений меньше 1000, то итерируем в пределах всей выборки
        sigma = 0
        dim2 = X.shape[1]
        for i in range(limit):
            for j in range (limit):
                sum = 0
                if i != j:
                    for k in range(dim2):
                        sum += (X[i][k] - X[j][k])**2
                    pares.append(sum)
                else:
                    pass
        sigma = median(pares)

        #создание новых признаков
        SE = sqrt(1/sigma)
        dim1 = X.shape[0]        
        self.B = np.random.uniform(low=-pi, high=pi, size=(1, self.n_features))[0].transpose()
        Xnew = np.zeros(shape=(X.shape[0], self.n_features)).tolist()
        for j in range(self.n_features):
            W = np.random.normal(loc=0, scale=SE, size=(1, dim2))[0]
            self.W.append(W)  
            for i in range(dim1):
                Xnew[i][j] = X[i].dot(W) + self.B[j]
        X = Xnew                                           
        
        #обучаем модель на полученных данных
        if self.svm == False:
            self.model = LogisticRegression(C=self.regularisation, tol=self.tolerance, max_iter=self.max_iter)
            self.model.fit(X = X, y = y)
        else:
            self.model = SVC(kernel=self.kernel, C=self.regularisation, tol=self.tolerance)
            self.model.fit(X = X, y = y)

        return self

        
    def predict(self, X):

        #адаптировать выборку
        if self.pca == True:
            X = self.pcamodel.transform(X)
        dim1 = X.shape[0] 
        Xnew = np.zeros(shape=(X.shape[0], self.n_features)).tolist()
        for j in range(self.n_features):
            for i in range(dim1):
                Xnew[i][j] = X[i].dot(self.W[j]) + self.B[j]
        X = Xnew                

        #применить модель
        if self.svm == False:
            y_pred = self.model.predict(X)
        else:
            y_pred = self.model.predict(X)

        return y_pred



In [0]:
Model = Taskone(svm=True)
Model.fit(X = x_train, y=y_train)

In [0]:
y_pred = Model.predict(X = x_test)

In [0]:
accuracy_score(y_test, y_pred)

0.8355

In [0]:
#как видим всё классно работает, ура

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг (используйте нормальную имплементацию, а не из sklearn, и подберите число деревьев и длину шага).

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [0]:
x_train_mini = x_train[0:5000][:]
y_train_mini = y_train[0:5000][:]
#вообще по-хорошему надо было взять хотя бы 15000-20000 наблюдений, но в своем юпитере у меня не получается открыть данные - что-то ему не нравится, а колаб постоянно 
#находит повод отключиться, пока модель обучается 25 раз

In [0]:
#SVM линейный на исходных:
#будем перебирать два основных гиперпараметра: C (соответствует линии в аутпуте - чем больше, тем ниже), tol (соответствует столбику в аутпуте - чем меньше, тем правее)
res = np.zeros(shape=(5,5)).tolist()
line = 0
vertic = 0
for i in ([1, 1.5, 2, 2.5, 3]):
    for j in ([1e-1, 1e-3, 1e-5, 1e-7, 1e-9]):
        SVMlinear = SVC(kernel='linear', C=i, tol=j)
        SVMlinear.fit(x_train_mini, y_train_mini)
        y_pred = SVMlinear.predict(X = x_test)
        res[line][vertic] = accuracy_score(y_test, y_pred)
        vertic += 1
    line += 1
    vertic = 0
    print(line)
res

1
2
3
4
5


[[0.7978, 0.7976, 0.7976, 0.7976, 0.7976],
 [0.7978, 0.7976, 0.7976, 0.7976, 0.7976],
 [0.7978, 0.7976, 0.7976, 0.7976, 0.7976],
 [0.7978, 0.7976, 0.7976, 0.7976, 0.7976],
 [0.7978, 0.7976, 0.7976, 0.7976, 0.7976]]

In [0]:
#SVM ядровой (гауссовское) на исходных:
#будем перебирать два основных гиперпараметра: C, tol
res = np.zeros(shape=(5,5)).tolist()
line = 0
vertic = 0
for i in ([1, 1.5, 2, 2.5, 3]):
    for j in ([1e-1, 1e-3, 1e-5, 1e-7, 1e-9]):
        SVMnonlin = SVC(C=i, tol=j)
        SVMnonlin.fit(x_train_mini, y_train_mini)
        y_pred = SVMnonlin.predict(X = x_test)
        res[line][vertic] = accuracy_score(y_test, y_pred)
        vertic += 1
    line += 1
    vertic = 0
    print(line)
res

1
2
3
4
5


[[0.8347, 0.8343, 0.8343, 0.8343, 0.8343],
 [0.8426, 0.8427, 0.8427, 0.8427, 0.8427],
 [0.8478, 0.8469, 0.8469, 0.8469, 0.8469],
 [0.8487, 0.8489, 0.8489, 0.8489, 0.8489],
 [0.8502, 0.8503, 0.8504, 0.8504, 0.8504]]

In [0]:
#наша модель, SVM ядровой (гауссовское):
#будем перебирать два основных гиперпараметра: C, tol
res = np.zeros(shape=(5,5)).tolist()
line = 0
vertic = 0
for i in ([1, 1.5, 2, 2.5, 3]):
    for j in ([1e-1, 1e-3, 1e-5, 1e-7, 1e-9]):
        SVMmodelnonlin = Taskone(svm=True, kernel='rbf', regularisation=i, tolerance=j)
        SVMmodelnonlin.fit(x_train_mini, y_train_mini)
        y_pred = SVMmodelnonlin.predict(X = x_test)
        res[line][vertic] = accuracy_score(y_test, y_pred)
        vertic += 1
    line += 1
    vertic = 0
    print(line)
res

1
2
3
4
5


[[0.7928, 0.7917, 0.7947, 0.7938, 0.7922],
 [0.8006, 0.803, 0.8024, 0.8027, 0.8022],
 [0.8083, 0.8066, 0.8066, 0.8083, 0.8068],
 [0.8116, 0.8106, 0.8106, 0.8125, 0.8134],
 [0.8139, 0.8133, 0.8143, 0.8157, 0.8146]]

In [0]:
#наша модель, SVM линейный:
#будем перебирать два основных гиперпараметра: C, tol
res = np.zeros(shape=(5,5)).tolist()
line = 0
vertic = 0
for i in ([1, 1.5, 2, 2.5, 3]):
    for j in ([1e-1, 1e-3, 1e-5, 1e-7, 1e-9]):
        SVMmodellinear = Taskone(svm=True, regularisation=i, tolerance=j)
        SVMmodellinear.fit(x_train_mini, y_train_mini)
        y_pred = SVMmodellinear.predict(X = x_test)
        res[line][vertic] = accuracy_score(y_test, y_pred)
        vertic += 1
    line += 1
    vertic = 0
    print(line)
res

1
2
3
4
5


[[0.8102, 0.8105, 0.808, 0.811, 0.8087],
 [0.8066, 0.8083, 0.81, 0.8077, 0.8088],
 [0.8087, 0.8084, 0.8083, 0.8082, 0.8081],
 [0.8083, 0.8079, 0.8073, 0.8065, 0.8105],
 [0.8075, 0.8073, 0.8088, 0.8068, 0.8088]]

In [0]:
import xgboost as xgb
pca = PCA(n_components=50)
pca.fit(x_train_mini)
x_train_forxgb = pca.transform(x_train_mini)
dtrain = xgb.DMatrix(x_train_forxgb, label=y_train_mini)
x_test_forxgb = pca.transform(x_test)
dtest = xgb.DMatrix(x_test_forxgb, label=y_test)

In [0]:
#как было сказано в задании, перебираем длинну шага (eta, соответсвует строкам в аутпуте - чем ниже, тем больше) и кол-во деревьев (соотв. столбикам, чем их больше, тем правее)
res = np.zeros(shape=(5,5)).tolist()
line = 0
vertic = 0
for i in ([0.1, 0.3, 0.5, 0.7, 1]):
    params = {'eta': i, 'objective': 'multi:softmax', 'num_class': 10}
    for j in ([50, 70, 90, 110, 130]):
        model_boosting = xgb.train(params=params, dtrain = dtrain, num_boost_round=j)
        y_pred = model_boosting.predict(dtest)
        res[line][vertic] = accuracy_score(y_test, y_pred)
        vertic += 1
    line += 1
    vertic = 0
    print(line)
res

1
2
3
4
5


[[0.8098, 0.8135, 0.8167, 0.8192, 0.8211],
 [0.8216, 0.8223, 0.8249, 0.8257, 0.8264],
 [0.8173, 0.8178, 0.8192, 0.8199, 0.8197],
 [0.8154, 0.8158, 0.8167, 0.8166, 0.8178],
 [0.8053, 0.8055, 0.8063, 0.8076, 0.8068]]

In [0]:
#ну в общем возможно на бОльших данных наша модель (как и бустинг) проявила бы себя лучше чем SVM, но как я написал выше, не представляется такое возможным.
#по результатам тестов лучший результат показал ядровой SVM, за ним - бустинг, затем - наша модель с ядровым SVM
#я забыл подрубать считалку времени и вспомнил о ней только после того, как уже всё прогнал :) но как я отчетливо помню, линейный SVM и бустинг обучались на порядок быстрее;
#ядровой SVM обучался сопоставимо долго с нашей моделью
#короче говоря если выбирать между нашей моделью и альтернативами, то бустинг во всём выиграл. *с учетом того, что данных мало взять получилось
#а еще я забыл указать рандом сиды, надеюсь это не отразится глобально на перфомансе моделей :)
#а еще про существование такой штуки как гридсерч я узнал вечером 29ого читая телегу, так что не судите строго

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

In [0]:
#1ое задание
Model = Taskone(pca=False, svm=True, kernel='rbf', regularisation=3, tolerance=1e-7)
Model.fit(X = x_train_mini.astype(int), y=y_train_mini)
y_pred = Model.predict(X = x_test)
accuracy_score(y_test, y_pred)

0.8176

In [0]:
#аналогичная модель (без МГК) была протестирована ранее выше перебором гиперпараметров и дала практически такой же результат на данных настройках
#возможно, понижение размерности не помогает из-за того, что объем данных невелик. тогда сравним с первоначальным тестом, на полной выборке, но без МГК:
Model = Taskone(pca=False, svm=True)
Model.fit(X = x_train.astype(int), y=y_train)
y_pred = Model.predict(X = x_test)
accuracy_score(y_test, y_pred)

#как видим и тут качество практически не изменилось (меняется кстати в лучшую сторону оба раза)

0.8411

In [0]:
#в общем я либо неправильно проверил (по хорошему надо было ведь так же наборы гиперпараметров смотреть, а не сравнить пару тестов с двумя старыми),
#либо понижение размерности при помощи PCA приводит лишь к ускорению алгоритма (обучение на полной выборке за пол часа с понижением против часа без) без изменения качества

In [0]:
#2ое задание:
x_train_mini = x_train[0:10000][:]
y_train_mini = y_train[0:10000][:]

res = []
for i in ([500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]):
    Model = Taskone(svm=True, kernel='rbf', n_features=i, regularisation=3, tolerance=1e-7)
    Model.fit(X = x_train_mini, y=y_train_mini)
    y_pred = Model.predict(X = x_test)
    res.append(accuracy_score(y_test, y_pred))
    print(i)
res

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


[0.8293, 0.8297, 0.8297, 0.829, 0.8304, 0.8286, 0.829, 0.8307, 0.8305, 0.8305]

In [0]:
res = []
for i in ([100, 150, 200, 250, 300, 350, 400, 450]):
    Model = Taskone(svm=True, kernel='rbf', n_features=i, regularisation=3, tolerance=1e-7)
    Model.fit(X = x_train_mini, y=y_train_mini)
    y_pred = Model.predict(X = x_test)
    res.append(accuracy_score(y_test, y_pred))
    print(i)
res

100
150
200
250
300
350
400
450


[0.8236, 0.8261, 0.826, 0.8253, 0.8298, 0.8279, 0.829, 0.8291]

In [0]:
#с учетом того, что наши фичи генерируются случайным образом, можно сказать что, независимо от от их количества, качество модели остается неизменным
#в целом да, с ростом количества фичей показатель качества начинает сходиться (в моем случае это было примерно 0.8305, но как писал раньше, я везде забыл зафиксировать рэндом сид),
#хотя и едва заметно - от малого числа до 5000 он возрос всего на 0.01 (даже немного меньше)

In [7]:
#3ее задание

#мы уже перебирали гиперпараметры для нашей модели для случая SVM. как показала практика, МГК приводит к значительному ускорению обучения и неизменному качеству;
#поэтому будем перебирать два основных гиперпараметра (как делали с SVM): C, tol
#используя при этом разложение МГК. Сравнивать будем с результатами ядрового SVM, так как у него было результаты лучше.
res = np.zeros(shape=(5,5)).tolist()
line = 0
vertic = 0
for i in ([1, 1.5, 2, 2.5, 3]):
    for j in ([1e-1, 1e-3, 1e-5, 1e-7, 1e-9]):
        SVMmodellog = Taskone(regularisation=i, tolerance=j)
        SVMmodellog.fit(x_train_mini.astype(int), y_train_mini)
        y_pred = SVMmodellog.predict(X = x_test)
        res[line][vertic] = accuracy_score(y_test, y_pred)
        vertic += 1
    line += 1
    vertic = 0
    print(line)
res

1
2
3
4
5


[[0.8203, 0.8211, 0.8201, 0.8211, 0.8205],
 [0.8206, 0.8194, 0.8204, 0.8211, 0.8206],
 [0.8206, 0.821, 0.8215, 0.8212, 0.8202],
 [0.8217, 0.8211, 0.8203, 0.8205, 0.8217],
 [0.8213, 0.8205, 0.8203, 0.8213, 0.8199]]

In [0]:
#так как данных было использовано в два раза больше, качество незначительно выше, но в целом модель с SVM отличается от модели с Лог регрессией только тем, 
#что имеет одинаковое значение качества независимо от гиперпараметров - с учетом того, что обучалась она так же долго, я бы отдал предпочтение ей, как более стабильной

### Бонус

Ниже приведены задания на бонусные баллы. За исследования на стандартных датасетах из sklearn или, скажем, на титанике, баллы не будут начисляться. Приветствуются интересные выводы из проведённых экспериментов.

__Задание 4. (Максимум 2 балла)__

Возьмите какой-нибудь достаточно сложный датасет, на котором хорошо работает градиентный бустинг. Сравните бустинг с нашим алгоритмом со случайными признаками с точки зрения качества и скорости. Подберите как следует гиперпараметры алгоритма.

__Задание 5. (Максимум 2 балла)__

Возьмите какой-нибудь датасет с текстами и решите одним из стандартных методов (например, tf-idf + логистическая регрессия или что-то нейросетевое). Сравните по качеству и скорости с нашим алгоритмом.

__Задание 6. (Максимум 2 балла)__

Поэкспериментируйте на нескольких датасетах из прошлых пунктов с фукнциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества.